<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/%EC%83%9D%EC%B2%B4_%EA%B4%91%ED%95%99_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Thu Jun 25 15:34:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [197]:
import torch
import torch.optim as optim
import pandas as pd
import numpy as np 
import torchvision.datasets as data
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [198]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [215]:
train_data = pd.read_csv('train.csv')
train_data = train_data.fillna(0)
train_data.head(20)

learning_rate = 0.001
training_epochs = 100
batch_size = 100
drop_prob = 0.5
#rho : 측정 거리 (단위: mm)
#src : 광원 스펙트럼 (650 nm ~ 990 nm)
#dst : 측정 스펙트럼 (650 nm ~ 990 nm)
#hhb : 디옥시헤모글로빈 농도
#hbo2 : 옥시헤모글로빈 농도
#ca : 칼슘 농도
#na : 나트륨 농도


In [216]:
x_data = train_data.iloc[: , 1:-4]
y_data = train_data.iloc[:,72:]
print(x_data)
print(y_data)

x_data = np.array(x_data)
y_data = np.array(y_data)
scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

      rho  650_src  660_src  ...       970_dst       980_dst       990_dst
0      25  0.37950  0.42993  ...  1.067504e-18  5.998949e-18  4.378513e-17
1      10  0.00000  0.00000  ...  0.000000e+00  9.710091e-09  0.000000e+00
2      25  0.00000  0.03289  ...  0.000000e+00  1.329725e-18  0.000000e+00
3      10  0.27503  0.31281  ...  0.000000e+00  4.088921e-10  0.000000e+00
4      15  1.01521  1.00872  ...  1.330237e-13  0.000000e+00  0.000000e+00
...   ...      ...      ...  ...           ...           ...           ...
9995   15  0.23929  0.30265  ...  9.690979e-16  1.391635e-15  5.460702e-14
9996   20  0.02583  0.00946  ...  0.000000e+00  5.825347e-16  4.226436e-15
9997   10  0.57589  0.62976  ...  0.000000e+00  1.504115e-10  4.003657e-10
9998   15  1.01477  1.01504  ...  0.000000e+00  2.818857e-12  1.408742e-11
9999   10  0.24452  0.28182  ...  1.219509e-11  3.259396e-12  3.830300e-11

[10000 rows x 71 columns]
        hhb  hbo2     ca    na
0      5.59  4.32   8.92  4.29
1      0.00

In [217]:
train_dataset = TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [218]:
linear1 = torch.nn.Linear(71, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 256, bias=True)
linear4 = torch.nn.Linear(256, 512, bias=True)
linear5 = torch.nn.Linear(512, 1024, bias=True)
linear6 = torch.nn.Linear(1024, 1024, bias=True)
linear7 = torch.nn.Linear(1024, 512, bias=True)
linear8 = torch.nn.Linear(512, 512, bias=True)
linear9 = torch.nn.Linear(512, 256, bias=True)
linear10 = torch.nn.Linear(256, 256, bias=True)
linear11 = torch.nn.Linear(256, 128, bias=True)
linear12 = torch.nn.Linear(128, 128, bias=True)
linear13 = torch.nn.Linear(128, 4, bias=True)
dropout = torch.nn.Dropout(p=drop_prob)
relu = torch.nn.ReLU()


In [219]:
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_normal_(linear6.weight)
torch.nn.init.xavier_normal_(linear7.weight)
torch.nn.init.xavier_normal_(linear8.weight)
torch.nn.init.xavier_uniform_(linear9.weight)
torch.nn.init.xavier_normal_(linear10.weight)
torch.nn.init.xavier_uniform_(linear11.weight)
torch.nn.init.xavier_normal_(linear12.weight)
torch.nn.init.xavier_normal_(linear13.weight)

model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5,relu,dropout,
                            linear6,relu,dropout,
                            linear7,relu,dropout,
                            linear8,relu,dropout,
                            linear9,relu,dropout,
                            linear10,relu,dropout,
                            linear11,relu,dropout,
                            linear12,relu,dropout,
                            linear13
                            ).to(device)

In [220]:
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

losses = []
model_history = []
err_history = []
total_batch = len(data_loader)

for epoch in range(training_epochs+1):
    avg_cost = 0
    for X,Y in data_loader:
      X = X.to(device)
      Y = Y.to(device)

      optimizer.zero_grad()
      hypothesis = model(X)
      cost = loss(hypothesis,Y)
      cost.backward()
      optimizer.step()
      avg_cost += cost / total_batch
    
    model_history.append(model)
    err_history.append(avg_cost)

    if epoch % 1 == 0:  
      print('Epoch:', '%d' % (epoch + 1), 'Cost =', '{:.9f}'.format(avg_cost))
    losses.append(cost.item())
print('Learning finished')

Epoch: 1 Cost = 13.483530998
Epoch: 2 Cost = 8.461858749
Epoch: 3 Cost = 7.475688457
Epoch: 4 Cost = 6.992487907
Epoch: 5 Cost = 6.864223480
Epoch: 6 Cost = 6.658472061
Epoch: 7 Cost = 6.514311314
Epoch: 8 Cost = 6.435469151
Epoch: 9 Cost = 6.382607937
Epoch: 10 Cost = 6.290222645
Epoch: 11 Cost = 6.284305096
Epoch: 12 Cost = 6.236523151
Epoch: 13 Cost = 6.210156441
Epoch: 14 Cost = 6.186075687
Epoch: 15 Cost = 6.109236717
Epoch: 16 Cost = 6.111011982
Epoch: 17 Cost = 6.101627827
Epoch: 18 Cost = 6.124692440
Epoch: 19 Cost = 6.124307632
Epoch: 20 Cost = 6.066931725
Epoch: 21 Cost = 6.022045135
Epoch: 22 Cost = 6.043918133
Epoch: 23 Cost = 6.038253307
Epoch: 24 Cost = 6.035329819
Epoch: 25 Cost = 6.036202431
Epoch: 26 Cost = 5.998600483
Epoch: 27 Cost = 6.001552105
Epoch: 28 Cost = 6.002033234
Epoch: 29 Cost = 5.993448734
Epoch: 30 Cost = 5.977298737
Epoch: 31 Cost = 5.983702183
Epoch: 32 Cost = 5.952881813
Epoch: 33 Cost = 5.961686134
Epoch: 34 Cost = 5.957734585
Epoch: 35 Cost = 5.930

In [221]:
best_model = model_history[np.argmin(err_history)]

In [222]:
xy_test = pd.read_csv('test.csv')
x_test = xy_test.iloc[:, 1:72]
x_test = x_test.fillna(0)
x_test = np.array(x_test)
x_test = scaler.transform(x_test)
x_test = torch.FloatTensor(x_test).to(device)

x_test[:3]

with torch.no_grad():
    model.eval()
    x_test=np.array(x_test.cpu())
    x_test=torch.from_numpy(x_test).float().to(device)
    predict=model(x_test)

In [223]:
predict

tensor([[7.9922, 3.9206, 8.9309, 2.9754],
        [7.9922, 3.9206, 8.9309, 2.9754],
        [7.9922, 3.9206, 8.9309, 2.9754],
        ...,
        [7.9922, 3.9206, 8.9309, 2.9754],
        [7.9922, 3.9206, 8.9309, 2.9754],
        [7.9922, 3.9206, 8.9309, 2.9754]], device='cuda:0')

In [224]:
submit = pd.read_csv('sample_submission.csv')
submit['hhb'] = submit['hhb'].astype(float)
submit['hbo2'] = submit['hbo2'].astype(float)
submit['ca'] = submit['ca'].astype(float)
submit['na'] = submit['na'].astype(float)
for i in range(len(predict)):
  submit['hhb'][i] = predict[i][0]
  submit['hbo2'][i] = predict[i][1]
  submit['ca'][i] = predict[i][2]
  submit['na'][i] = predict[i][3]

submit.to_csv('submit.csv', mode = 'w', index = False, header = True)
submit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: Setti

,id,hhb,hbo2,ca,na
0,10000,7.992213,3.920603,8.930861,2.975384
1,10001,7.992213,3.920603,8.930861,2.975384
2,10002,7.992213,3.920603,8.930861,2.975384
3,10003,7.992213,3.920603,8.930861,2.975384
4,10004,7.992213,3.920603,8.930861,2.975384
...,...,...,...,...,...
9995,19995,7.992213,3.920603,8.930861,2.975384
9996,19996,7.992213,3.920603,8.930861,2.975384
9997,19997,7.992213,3.920603,8.930861,2.975384
9998,19998,7.992213,3.920603,8.930861,2.975384
